In [0]:
from PIL import Image
import numpy as np
import pickle
import random

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My Drive/flipchal

/content/drive/My Drive/flipchal


In [0]:
lsF

id_to_box   id_to_data2  id_to_size3  test1.csv     test.gsheet
id_to_box2  id_to_data3  id_to_size4  test1.gsheet  test_set/
id_to_box3  id_to_data4  model2.h5    test2.csv     training.csv
id_to_box4  id_to_size   model.h5     test3.csv     train_set/
id_to_data  id_to_size2  p1/          test.csv      Untitled.ipynb


In [0]:
import numpy as np

In [0]:
def Normalize(image,mean,std):
    for channel in range(3):
        image[:,:,channel]=(image[:,:,channel]-mean[channel])/std[channel]
    return image

## Skip the next few ones if you have processed the data already

In [0]:
id_to_data={}
id_to_size={}

In [0]:
with open("./data/images.txt") as f:
    lines=f.read().splitlines()
    for line in lines:
        id,path=line.split(" ",1)
        if int(id) <4000:
            image=Image.open("./data/images/"+path).convert('RGB')
            id_to_size[int(id)]=np.array(image,dtype=np.float32).shape[0:2]
            image=image.resize((224,224))
            image=np.array(image,dtype=np.float32)
            image=image/255
            image=Normalize(image,[0.485,0.456,0.406],[0.229,0.224,0.225])
            id_to_data[int(id)]=image
            if (int(id)%500)==0:
                print('Pic no '+ id+' ')

In [0]:
id_to_data=np.array(list(id_to_data.values()))
id_to_size=np.array(list(id_to_size.values()))

In [0]:
id_to_box={}

In [0]:
with open("./data/bounding_boxes.txt") as f:
    lines=f.read().splitlines()
    for line in lines:
        id,box=line.split(" ",1)
        if int(id) <4000:
          box=np.array([float(i) for i in box.split(" ")],dtype=np.float32)
          box[0]=box[0]/id_to_size[int(id)-1][1]*224
          box[1]=box[1]/id_to_size[int(id)-1][0]*224
          box[2]=box[2]/id_to_size[int(id)-1][1]*224
          box[3]=box[3]/id_to_size[int(id)-1][0]*224
          id_to_box[int(id)]=box
          if (int(id)%500)==0:
              print('Pic no '+ id+' ')

Pic no 500 
Pic no 1000 
Pic no 1500 
Pic no 2000 
Pic no 2500 
Pic no 3000 
Pic no 3500 


In [0]:
f=open("./id_to_data","wb+")
pickle.dump(id_to_data,f,protocol=2)
print('ID data file written')
f=open("./id_to_size","wb+")
pickle.dump(id_to_size,f,protocol=2)

In [0]:
id_to_box=np.array(list(id_to_box.values()))

In [0]:
def getdata():
    # read data and shuffle
    index=[i for i in range(3499)]
    random.shuffle(index)
    
    f = open("./id_to_data","rb+")
    data = pickle.load(f)
    data=data[index]
    data_train=data[0:3499]

    f = open("./id_to_box","rb+")
    box = pickle.load(f)
    box=box[index]
    box_train=box[0:3499]
    
    return data_train,box_train

In [0]:
def plot_model(model_details):
    fig, axs = plt.subplots(1,2,figsize=(15,5))

    axs[0].plot(range(1,len(model_details.history['my_metric'])+1),model_details.history['my_metric'])
    axs[0].plot(range(1,len(model_details.history['val_my_metric'])+1),[1.7*x for x in model_details.history['val_my_metric']])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_details.history['my_metric'])+1),len(model_details.history['my_metric'])/10)
    axs[0].legend(['train', 'val'], loc='best')

    axs[1].plot(range(1,len(model_details.history['loss'])+1),model_details.history['loss'])
    axs[1].plot(range(1,len(model_details.history['val_loss'])+1),model_details.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_details.history['loss'])+1),len(model_details.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')

    plt.savefig("model.png")

In [0]:
import tensorflow as tf
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, MaxPooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
data_train,box_train=getdata()

In [0]:
box_train

array([[ 41.3     ,  46.2     , 149.45    ,  88.66667 ],
       [ 18.199999,  83.066666, 201.59999 ,  60.666668],
       [ 43.05    ,  51.8     , 171.84999 , 109.66667 ],
       ...,
       [ 54.600002,  21.      , 116.90001 , 158.66666 ],
       [ 18.550001,  49.      , 143.15001 , 111.53333 ],
       [ 17.5     ,  79.799995, 189.7     ,  45.733334]], dtype=float32)

In [0]:
def my_metric(labels,predictions):
    threshhold=0.75
    x=predictions[:,0]*224
    x=tf.maximum(tf.minimum(x,224.0),0.0)
    y=predictions[:,1]*224
    y=tf.maximum(tf.minimum(y,224.0),0.0)
    width=predictions[:,2]*224
    width=tf.maximum(tf.minimum(width,224.0),0.0)
    height=predictions[:,3]*224
    height=tf.maximum(tf.minimum(height,224.0),0.0)
    label_x=labels[:,0]
    label_y=labels[:,1]
    label_width=labels[:,2]
    label_height=labels[:,3]
    a1=tf.multiply(width,height)
    a2=tf.multiply(label_width,label_height)
    x1=tf.maximum(x,label_x)
    y1=tf.maximum(y,label_y)
    x2=tf.minimum(x+width,label_x+label_width)
    y2=tf.minimum(y+height,label_y+label_height)
    IoU=tf.abs(tf.multiply((x1-x2),(y1-y2)))/(a1+a2-tf.abs(tf.multiply((x1-x2),(y1-y2))))
    condition=tf.less(threshhold,IoU)
    sum=tf.where(condition,tf.ones(tf.shape(condition)),tf.zeros(tf.shape(condition)))
    return tf.reduce_mean(sum)

In [0]:
def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*224)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)

In [0]:
def resnet_block(inputs,num_filters,kernel_size,strides,activation='relu'):
    x=Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(inputs)
    x=BatchNormalization()(x)
    if(activation):
        x=Activation('relu')(x)
    return x

In [0]:
def resnet18():
    inputs=Input((224,224,3))
    
    # conv1
    x=resnet_block(inputs,64,[7,7],2)

    # conv2
    x=MaxPooling2D([3,3],2,'same')(x)
    for i in range(2):
        a=resnet_block(x,64,[3,3],1)
        b=resnet_block(a,64,[3,3],1,activation=None)
        x=keras.layers.add([x,b])
        x=Activation('relu')(x)
    
    # conv3
    a=resnet_block(x,128,[1,1],2)
    b=resnet_block(a,128,[3,3],1,activation=None)
    x=Conv2D(128,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,128,[3,3],1)
    b=resnet_block(a,128,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    # conv4
    a=resnet_block(x,256,[1,1],2)
    b=resnet_block(a,256,[3,3],1,activation=None)
    x=Conv2D(256,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,256,[3,3],1)
    b=resnet_block(a,256,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    # conv5
    a=resnet_block(x,512,[1,1],2)
    b=resnet_block(a,512,[3,3],1,activation=None)
    x=Conv2D(512,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,512,[3,3],1)
    b=resnet_block(a,512,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    x=AveragePooling2D(pool_size=7,data_format="channels_last")(x)
    # out:1*1*512

    y=Flatten()(x)
    # out:512
    y=Dense(1000,kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(y)
    outputs=Dense(4,kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(y)
    
    model=Model(inputs=inputs,outputs=outputs)
    return model

In [0]:
model = resnet18()
model.compile(loss=smooth_l1_loss,optimizer=Adam(),metrics=[my_metric])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
_____________________________________

In [0]:
def lr_sch(epoch):
    #200 total
    if epoch <100:
        return 1e-4
    if epoch>=100:
        return 1e-5

In [0]:
lr_scheduler=LearningRateScheduler(lr_sch)
lr_reducer=ReduceLROnPlateau(monitor='val_my_metric',factor=0.2,patience=5,mode='max',min_lr=1e-3)

In [0]:
model=keras.models.load_model("./model2.h5", custom_objects={'smooth_l1_loss':smooth_l1_loss, 'my_metric':my_metric})

In [0]:
checkpoint=ModelCheckpoint('model.h5',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
model_details=model.fit(data_train,box_train,batch_size=32,epochs=20,shuffle=True,validation_split=0.3,callbacks=[lr_scheduler,lr_reducer,checkpoint],verbose=1)
model.save('model2.h5')

Train on 2449 samples, validate on 1050 samples
Epoch 1/20
2449/2449 [==============================] - 31s 13ms/step - loss: 32.5413 - my_metric: 0.8987 - val_loss: 31.5849 - val_my_metric: 0.9000
Epoch 2/20
2449/2449 [==============================] - 24s 10ms/step - loss: 29.4787 - my_metric: 0.9151 - val_loss: 31.3078 - val_my_metric: 0.8971


KeyboardInterrupt: ignored

In [0]:
model.save('model2.h5')

In [0]:
import matplotlib.pyplot as plt


In [0]:
plt.switch_backend('agg')

In [0]:
index=[i for i in range(500)]
index=random.sample(index,100)

In [0]:
model=keras.models.load_model("./model2.h5", custom_objects={'smooth_l1_loss':smooth_l1_loss, 'my_metric':my_metric})
result=model.predict(data_train[0:1,:,:,:])

mean=[0.485,0.456,0.406]
std=[0.229,0.224,0.225]
j=0

In [0]:
data_train

array([[[[ 1.0330508 ,  0.3977592 , -0.09638336],
         [ 0.9988013 ,  0.3802522 , -0.07895417],
         [ 0.9816765 ,  0.3627452 , -0.06152498],
         ...,
         [ 0.96455175,  0.87044823,  0.8970808 ],
         [ 0.9988013 ,  0.90546227,  0.9319392 ],
         [ 1.0330508 ,  0.94047624,  0.9667976 ]],

        [[ 0.9988013 ,  0.3977592 , -0.11381256],
         [ 0.9988013 ,  0.3802522 , -0.07895417],
         [ 0.96455175,  0.3802522 , -0.09638336],
         ...,
         [ 0.9988013 ,  0.90546227,  0.8970808 ],
         [ 1.015926  ,  0.9229692 ,  0.91451   ],
         [ 1.0330508 ,  0.94047624,  0.9319392 ]],

        [[ 0.96455175,  0.3802522 , -0.13124175],
         [ 0.9988013 ,  0.41526622, -0.06152498],
         [ 0.96455175,  0.3802522 , -0.06152498],
         ...,
         [ 1.015926  ,  0.9229692 ,  0.91451   ],
         [ 1.015926  ,  0.9229692 ,  0.91451   ],
         [ 1.0501755 ,  0.95798326,  0.94936836]],

        ...,

        [[ 1.6666666 ,  1.6232493 ,  1

In [0]:
for i in range(1):
    print("Predicting "+str(i)+"th image.")
    true_box=box_train[i]
    image=data_train[i]
    prediction=result[j]
    j+=1
    for channel in range(3):
        image[:,:,channel]=image[:,:,channel]*std[channel]+mean[channel]

    image=image*255
    image=image.astype(np.uint8)
    plt.imshow(image)


    plt.gca().add_patch(plt.Rectangle((true_box[0],true_box[1]),true_box[2],true_box[3],fill=False,edgecolor='red',linewidth=2,alpha=0.5))
    plt.gca().add_patch(plt.Rectangle((prediction[0]*224,prediction[1]*224),prediction[2]*224,prediction[3]*224,fill=False,edgecolor='green',linewidth=2,alpha=0.5))
    
    plt.show()
    plt.savefig("./p1/"+str(i)+".png")
    print(str(prediction[0]*224)+' '+str(prediction[1]*224)+' '+str(prediction[2]*224)+' '+str(prediction[3]*224)+' ')
    plt.cla()


Predicting 0th image.
56.83512592315674 7.019866943359375 113.93784141540527 174.57539749145508 
